<h1>Knap Sack 0/1 problem using genetic algorithm</h1>

In [ ]:
import random

items = [
    (1, 14),
    (3, 23),
    (7, 8),
    (4, 9),
    (5, 17),
    (6, 15)
]

capacity = 10
n = len(items)
pop_size = 8
max_gen = 100
mut_prob = 0.1

def create_individual():
    return [random.randint(0, 1) for _ in range(n)]

def initialize_population():
    return [create_individual() for _ in range(pop_size)]

def evaluate(individual):
    total_weight = 0
    total_value = 0
    for i in range(len(individual)):
        if individual[i] == 1:
            total_weight += items[i][0]
            total_value += items[i][1]
    if total_weight > capacity:
        return 0 #total_value - total_weight
    else:
        return total_value

def select_two_best_by_probability(population, fitnesses):
    min_fit = min(fitnesses)
    if min_fit < 0:
        adjusted_fitness = [f - min_fit + 1 for f in fitnesses]
    else:
        adjusted_fitness = fitnesses

    total_fit = sum(adjusted_fitness)
    probabilities = [f / total_fit for f in adjusted_fitness]
    
    sorted_indices = sorted(range(len(probabilities)), key=lambda i: probabilities[i], reverse=True)
    
    best_index = sorted_indices[0]
    second_best_index = sorted_indices[1]

    return population[best_index], population[second_best_index]

def crossover(parent1, parent2):
    point = random.randint(1, n - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(individual):
    for i in range(n):
        if random.random() < mut_prob:
            individual[i] = 1 - individual[i]
    return individual

def replace_population(population, offspring):
    fitnesses = [evaluate(ind) for ind in population]
    sorted_indices = sorted(range(len(population)), key=lambda i: fitnesses[i])
    for i, child in zip(sorted_indices[:len(offspring)], offspring):
        population[i] = child
    return population

def genetic_algorithm():
    population = initialize_population()
    best_solution = None
    best_fitness = float('-inf')
    for gen in range(max_gen):
        fitnesses = [evaluate(ind) for ind in population]
        for ind, fit in zip(population, fitnesses):
            if fit > best_fitness:
                best_fitness = fit
                best_solution = ind
        parent1, parent2 = select_two_best_by_probability(population, fitnesses)
        child1, child2 = crossover(parent1, parent2)
        child1 = mutate(child1)
        child2 = mutate(child2)
        print("Generation", gen + 1, "offspring:")
        print("Child 1:", child1)
        print("Child 2:", child2)
        print("-" * 40)
        population = replace_population(population, [child1, child2])
    return best_solution, best_fitness

def print_solution(individual):
    total_weight = sum(gene * items[i][0] for i, gene in enumerate(individual))
    total_value = sum(gene * items[i][1] for i, gene in enumerate(individual))
    print("Best solution:")
    print("Chromosome:", individual)
    print("Total weight:", total_weight)
    print("Total value:", total_value)

best_ind, best_fit = genetic_algorithm()
print_solution(best_ind)



import random
import string

# Global parameters
population_size = 100
mutation_rate = 0.01
crossover_rate = 0.7
max_generations = 5000
character_pool = string.ascii_uppercase + " "  # Uppercase letters + space

# Helper function to generate a random string of a given length.
def random_string(length):
    return ''.join(random.choice(character_pool) for _ in range(length))

# Roulette-wheel selection: choose a parent with probability proportional to its fitness.
def select_parent(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    if total_fitness == 0:
        return random.choice(population)
    r = random.uniform(0, total_fitness)
    cumulative = 0
    for individual, score in zip(population, fitness_scores):
        cumulative += score
        if cumulative >= r:
            return individual

# Single-point crossover between two parents.
def crossover(parent1, parent2, crossover_rate):
    if random.random() < crossover_rate:
        point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2
    else:
        return parent1, parent2

# Mutation: randomly change each character with probability mutation_rate.
def mutate(individual, mutation_rate):
    mutated = list(individual)
    for i in range(len(mutated)):
        if random.random() < mutation_rate:
            mutated[i] = random.choice(character_pool)
    return ''.join(mutated)

# Run the genetic algorithm for a given target string.
def run_ga(target):
    # Fitness function: number of characters matching the target in the correct position.
    def fitness(individual):
        return sum(1 for i, j in zip(individual, target) if i == j)
    
    # Initialize population with random strings.
    population = [random_string(len(target)) for _ in range(population_size)]
    generation = 0

    while generation < max_generations:
        #fitness_scores = [fitness(ind) for ind in population]
        fitness_scores=[]
        for ind in population:
            fitness_scores.append(fitness(ind))
        best_individual = max(population, key=fitness)
        best_fitness = fitness(best_individual)
        
        # Debug: print progress every 100 generations.
        if generation % 100 == 0:
            avg_fitness = sum(fitness_scores) / len(fitness_scores)
            print(f"Target: '{target}'")
            print(f"Generation {generation} | Best fitness: {best_fitness} | Avg fitness: {avg_fitness:.2f} | Best individual: '{best_individual}'")
        
        # Stop if the optimal solution is found.
        if best_fitness == len(target):
            print(f"\nSolution found for '{target}' in generation {generation}: '{best_individual}'")
            break
        
        # Create a new generation.
        new_population = []
        while len(new_population) < population_size:
            parent1 = select_parent(population, fitness_scores)
            parent2 = select_parent(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2, crossover_rate)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            new_population.extend([child1, child2])
        
        # Ensure population size remains constant.
        population = new_population[:population_size]
        generation += 1

    if generation == max_generations and best_fitness != len(target):
        print(f"\nMax generations reached for '{target}'. Best individual: '{best_individual}' with fitness {best_fitness}")
    
    print("-----------------------------------------------------\n")

# List of target strings to run the genetic algorithm on.
targets = [
    "TOB TOB TABALI",
    # "EVOLUTIONARY ALGORITHM",
    # "NATURAL SELECTION",
    "SIGMA SIGMA BOY"
    # "ARTIFICIAL INTELLIGENCE",
    # "NATURAL STUPDITY"
]

# Run the GA for each target.
for target in targets:
    run_ga(target)



Generation 1 offspring:
Child 1: [1, 0, 0, 0, 1, 0]
Child 2: [0, 0, 0, 0, 0, 1]
----------------------------------------
Generation 2 offspring:
Child 1: [0, 0, 0, 0, 0, 1]
Child 2: [0, 0, 1, 0, 1, 0]
----------------------------------------
Generation 3 offspring:
Child 1: [1, 0, 0, 1, 0, 1]
Child 2: [1, 0, 0, 0, 1, 0]
----------------------------------------
Generation 4 offspring:
Child 1: [1, 0, 1, 0, 1, 0]
Child 2: [1, 0, 1, 1, 1, 0]
----------------------------------------
Generation 5 offspring:
Child 1: [1, 0, 0, 0, 1, 0]
Child 2: [1, 0, 0, 0, 1, 0]
----------------------------------------
Generation 6 offspring:
Child 1: [1, 0, 0, 0, 1, 0]
Child 2: [1, 0, 0, 0, 1, 0]
----------------------------------------
Generation 7 offspring:
Child 1: [1, 0, 0, 1, 1, 0]
Child 2: [1, 0, 0, 1, 0, 1]
----------------------------------------
Generation 8 offspring:
Child 1: [1, 0, 0, 1, 1, 0]
Child 2: [1, 0, 0, 1, 0, 0]
----------------------------------------
Generation 9 offspring:
Child 1: